In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Input, BatchNormalization,\
      Conv2D, Add, ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Reshape
from keras.initializers import glorot_uniform


2021-12-23 22:37:24.848086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-23 22:37:24.848121: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(f, f), strides=(s, s), padding='same', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)

    X_shortcut = Conv2D(filters=F2, kernel_size=(f, f), strides=(s, s), padding='same', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [7]:
def SameResNet(input_shape=(8, 8, 8)):

    X_input = Input((1, ) + input_shape)

    X = Reshape(target_shape= input_shape)(X_input)

    X = ZeroPadding2D((2, 2))(X)

    X = Conv2D(64, (8, 8), strides=(1, 1), padding='same', name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64], stage=2, block='b')
    X = identity_block(X, 3, [64, 64], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128], stage=3, block='b')
    X = identity_block(X, 3, [128, 128], stage=3, block='c')
    X = identity_block(X, 3, [128, 128], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256], stage=4, block='a', s=1)
    X = identity_block(X, 3, [256, 256], stage=4, block='b')
    X = identity_block(X, 3, [256, 256], stage=4, block='c')
    X = identity_block(X, 3, [256, 256], stage=4, block='d')
    X = identity_block(X, 3, [256, 256], stage=4, block='e')
    X = identity_block(X, 3, [256, 256], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512], stage=5, block='b')
    X = identity_block(X, 3, [512, 512], stage=5, block='c')

    X = GlobalAveragePooling2D()(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [8]:
def get_model888(STATE_SHAPE = (8, 8, 8), NB_ACTIONS = 4096):
    base_model = SameResNet(input_shape = STATE_SHAPE)
    
    headModel = base_model.output
    headModel = Flatten()(headModel)
    headModel = Dense(1000,activation='softmax', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
    headModel = Dense(NB_ACTIONS,activation='linear', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

    model = Model(inputs=base_model.input, outputs=headModel)
    return model

In [9]:
model = get_model888()
model.summary()

2021-12-23 22:38:05.158142: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-23 22:38:05.158191: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-23 22:38:05.158241: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist
2021-12-23 22:38:05.158733: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 8, 8, 8)] 0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 8, 8, 8)      0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 12, 12, 8)    0           reshape_1[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 12, 12, 64)   32832       zero_padding2d_1[0][0]           
______________________________________________________________________________________________